# Recursive Backtracking

# Dinner Invitation Problem

You have a wide social circle and like entertaining. Unfortunately, not all of your friends like each other, in fact, some dislike others intensely. When you throw a dinner party, you want to make sure that no fights are going to break out. And so you don’t want to invite any pair of friends who are liable to get into fisticuffs or heated arguments and spoil the party.  The more the merrier, so you want to invite as many friends as possible.

To get a sense of what you are up against, let’s represent your social circle pictorially as a graph. Each vertex of the graph is one of your friends. The graph also has edges between vertices.  If Alice dislikes Bob, then we will have an edge between the Alice vertex and the Bob vertex.  Note that Bob may (secretly) like Alice or the dislike may be mutual, but either way, you don’t want Alice and Bob together at your house. Think of the edge between the Alice and Bob vertices signifying that one of them does not like the other or that both dislike each other.

For example, suppose that your social circle looks like this:

<img width="50%" src="img/dinner_graph.png">

The total number of possible guests is five. However, there are two edges in the above graph representing “dislike” relationships. You can’t invite Alice and Bob together, and you can’t invite Bob and Eve. There is no transitivity in the dislikes relationship — Alice dislikes Bob who dislikes Eve, but Alice is just fine with Eve and Eve with Alice in the example above. If it is the case that Eve dislikes Alice, or vice versa, there will be an edge between Eve and Alice. Likes or dislikes are unpredictable, just as in real-life social circles!

Cleo and Don are obvious choices since they don’t preclude anyone else from being invited. If you invite Bob, you can’t invite either Alice or Eve. But if you don’t invite Bob, you can invite four of your friends: Cleo, Don, Alice and Eve. That is maximum number of people you can have over for dinner in your social circle.

> **Problem:** Given an arbitrarily complex social circle (e.g., graph below), can you think of an algorithm that always finds a maximum number of mutually friendly people to invite for dinner?

<div style="text-align: center"><img style="display:inline-block" width="40%" src="img/graph_unsolved.png"> $\quad\rightarrow\quad$ <img style="display:inline-block" width="40%" src="img/graph_solved.png"></div>

## First Attempt: "Greedy"

We’ll first try what is called a greedy approach. A greedy algorithm for our puzzle would pick the guest with the least number of dislikes, i.e., the one with the fewest number of connecting edges. Once this guest is picked, all guests disliked by this guest are eliminated from consideration. The process continues until all guests are either selected or eliminated.

In our first example, the greedy algorithm would pick Cleo and Don (no connecting edges). No guests are eliminated. Then, it would pick Alice or Eve who each have one connecting edge, since Bob has two connecting edges. Picking either Alice or Eve eliminates Bob. The guest that was not picked remains and can be picked. A possible sequence of greedy selections resulting in the maximum selection is shown below.

<img width="50%" src="img/dinner_graph_ordered.png">

Now consider the example below. There are several vertices/guests with two edges, and the greedy algorithm could easily pick C.

<img width="50%" src="img/greedy_is_bad.png">

If the greedy algorithm picks C, then B and D are eliminated. After that it can only pick one of A, E, or F producing a solution containing two guests. However, observe that the maximum selection has three guests, namely, B, D, and F. 

Thus this greedy algorithm does not always work for our problem. In fact, our dinner problem is a classic problem called the **maximum independent set (MIS)** problem: Given a graph with vertices and edges, find a maximum set of vertices that do not have edges between them.  There are several situations where we might have to solve MIS. For example, suppose a franchise is trying to identify new locations such that no two locations are close enough to compete with each other.   Construct a graph where the vertices are possible locations, and add edges between any two locations deemed close enough to interfere. The MIS gives you the maximum number of locations you can sell without cannibalizing sales.

MIS is a hard problem ("**NP-complete**"): every known algorithm that guarantees the maximum cardinality of chosen candidates for all problem instances, including the algorithm we coded, takes time exponential in the number of guests for some instances. If someone came up with an efficient algorithm to produce the maximum selection in all independent set problem instances, whose runtime grows polynomially in the number of guests, or was able to formally prove that no such algorithm exists, that person would solve one of the remaining unsolved Millennium Prize Problems (P = NP), would win a prize of US$\$$1 million, and more important, attain instant rock-star status in Computer Science. ("Polynomially" means that the runtime grows as $n^k$, where $n$ is the number of candidates, and $k$ is a fixed constant.  More in 6.006.)


## Recursive Backtracking

We used recursive enumeration in the N-queens problem in Lecture 1. The enumeration procedure checks for conflicts immediately after a queen is placed on the board. This is helpful in improving performance — we do not want to extend a configuration that already has a conflict. Adding queens to this configuration will not result in a valid solution. Let’s use recursion to solve the dinner problem in a similar way.

We will recursively generate combinations of guests to invite and perform early conflict detection so we don’t extend bad combinations. We start with an empty combination and an available guest list that is initially equal to the list of guests. In our recursive strategy, we have two branches in the recursion:

1.	We add a new guest from the available guest list to the current combination and recur only if the combination remains valid.

2.	We eliminate a guest from the available guest list without adding the guest to the current combination and recur.

The base case is when the available guest list is empty. During the recursion, we have to maintain the best, i.e., maximum cardinality solution found.

Suppose we have a guest list `[A, B, C]` and the single dislike relationship `[A, B]`. Here’s what the entire recursive execution (called the recursion tree) looks like. `chosen` corresponds to the current combination, i.e., the currently chosen guests, and `vertices` corresponds to the guests available to be chosen.

<img width="50%" src="img/backtracking.png">

The key observation about the execution above is that a conflict results in the termination of the branch. At the bottom we have the base cases, and the two maximum cardinality solutions `[B, C]` and `[A, C]`. One of these will be picked depending on the order that the tree is executed, i.e., whether the yes branch is executed first, or the no branch.

Here is an implementation of this recursion:

In [1]:
def MIS(vertices, edges, chosen = [], best = []):
    """Find a maximum independent set in a given graph.
    vertices is a list of vertices still available to be chosen.
    edges is a list of pairs of vertices that should not be chosen together.
    chosen is the list of currently chosen vertices in backtracking search.
    best is the best solution found so far.
    """
    if len(vertices) == 0:
        if len(chosen) > len(best):
            best = chosen
        return best
    if independent(chosen + [vertices[0]], edges):
        best = MIS(vertices[1:], edges, chosen + [vertices[0]], best)
    return MIS(vertices[1:], edges, chosen, best) 

The base case (Line 8) is that the list of available guests is empty. If the current solution `chosen` is larger than the previous best solution `best` (e.g., `best` is empty), we update `best`. In either case, we return `best`.

Lines 12-13 correspond to Case 1 of the recursion. Line 12 checks whether adding the first guest `vertices[0]` in the available list to `chosen` results in a conflict. If it does not, we add `vertices[0]` to `chosen`, remove it from the available guest list by list slicing and recurse (Line 13). Line 14 corresponds to Case 2 of the recursion, where we recurse without adding `vertices[0]` to `chosen`.

The procedure `independent` goes through the dislike relationships and checks whether two guests who dislike each other are both in the invite list:

In [2]:
def independent(chosen, badPairs):
    """Are the chosen vertices independent, avoiding all bad pairs?"""
    for a, b in badPairs:
        if a in chosen and b in chosen:
            return False
    return True

In [3]:
MIS(["A","B","C"], [["A","B"]])

['A', 'C']

In [4]:
MIS(["Alice","Bob","Cleo","Don","Eve"],
    [["Alice","Bob"], ["Bob","Eve"]])

['Alice', 'Cleo', 'Don', 'Eve']

In [5]:
MIS(["A","B","C","D","E","F"],
    [["A","B"], ["B","C"], ["C","D"], ["D","E"], ["A","E"], ["E","F"], ["F","A"]])

['B', 'D', 'F']

In [6]:
MIS([f"i{k}" for k in range(12)] + [f"o{k}" for k in range(12)],
    [[f"i{k}", f"o{k}"] for k in range(12)] +
    [[f"o{k}", f"o{(k+1)%12}"] for k in range(12)] +
    [[f"i{k}", f"i{(k+4)%12}"] for k in range(12)]
)

['i0', 'i1', 'i3', 'i6', 'o2', 'o4', 'o7', 'o9', 'o11']

# Solving Sudoku

[Sudoku](https://en.wikipedia.org/wiki/Sudoku) is a popular number-placement puzzle. The objective is to fill a partially filled 9 × 9 grid with digits so that each column, each row, and each of the nine 3 × 3 subgrids or sectors that compose the grid contains all of the digits from 1 to 9.

These constraints are used to determine the missing numbers.  In the puzzle below, several subgrids have missing numbers.  Scanning rows (or columns as the case may be) can tell us where to place a missing number in a sector.  For example, we can determine the position of the 8 in the top middle sector, because 8 *cannot* be placed in the middle or bottom rows of the top middle sector.

<div style="text-align:center"><img style="display:inline-block" width="40%" src="img/sudoku1.png">$\qquad$<img style="display:inline-block" width="40%" src="img/sudoku2.png"></div>

Our goal is to write a Sudoku solver that can do a recursive search for numbers to be placed in the missing positions, regardless of how many numbers are filled in.  The basic solver does not follow a human strategy like the one described above. It guesses a number at a particular location and determines if the guess violates a constraint or not.  If not, it proceeds to guess other numbers at other positions.  If a violation is detected, then the most recent guess is changed.  This is similar to the N-queens search of Lecture 1.  Once we develop this basic recursive backtracking strategy, we will add some “human intelligence” to the solver using observations like those above.

## Recursive Sudoku Solving

Below is the top-level routine for a basic recursive Sudoku solver. You can find the code in `sudoku.py`. The grid is represented by a two-dimensional array/list called `grid` and a value of `0` means that the location is empty. Grid locations are filled in through a process of a systematic ordered search for empty locations, guessing values for each location, and backtracking, i.e., undoing guesses if they are incorrect.

In [7]:
backtracks = 0

def solveSudoku(grid):
    """Try to solve a Sudoku puzzle in-place, and return whether successful.
    grid is a list of lists of integers between 0 and 9, where 0 means unfilled,
    and each top-level list corresponds to a row of 9 cells."""
    global backtracks
    i, j = findNextCellToFill(grid)
    if i == -1: # no more empty cells
        return True
    for e in range(1, 10): # 1..9
        if isValid(grid, i, j, e):
            grid[i][j] = e
            if solveSudoku(grid):
                return True
            backtracks += 1
            grid[i][j] = 0
    return False

Procedure `findNextCellToFill`, which will be shown and explained later, finds the first empty (value `0`) by searching the grid in a predetermined order. If the procedure cannot find an empty value, the puzzle is solved. 

Procedure `isValid`, which will also be shown and explained later, checks that the current grid that is partially filled in does not violate the rules of Sudoku.  This is reminiscent of `noConflicts` in the N-queens puzzle that also worked with partial configurations (i.e., configurations with fewer than N queens) as well as `independent` in the example above.

The first important point about `solveSudoku` is that there is only one copy of `grid` that is being operated on and modified. `solveSudoku` is therefore an in-place recursive search exactly like N-queens.  Because of this, we have to change back the value of the position that was filled in with an incorrect number (Line 13) back to `0` (Line 17) after the recursive call for a particular guess returns `False` and the loop continues. This is called the **do-undo paradigm** for recursive backtracking: we make a change, recurse, and then undo the change. Note that Line 17 could be moved out of the `for` loop, since the `grid[i][j]` value is overwritten on each iteration of the loop. However, after we exit the loop, before returning `False`, we need to clear the grid entry.

One programming construct that you might not have seen before is `global`. Global variables retain state across function invocations and are convenient to use when we want to keep track of how many recursive calls are made, etc. We use `backtracks` as a global variable, initially setting to zero (at the top of the file), and incrementing it each time we realize we have made an incorrect guess that we need to undo. Note that in order to use `backtracks` in `sudokuSolve` we have to declare it global within the function.

Computing the number of backtracks is a great way of measuring performance independent of the computing platform. The larger number of backtracks, the longer the program typically takes to run. 

Now, let’s take a look at the procedures invoked by `solveSudoku`.

`findNextCellToFill` follows a prescribed order in searching for an empty location, going column by column, starting with the leftmost column and moving rightward. Any order can be used as long as we ensure that we will not miss any empty values in the current grid at any point in the recursive search. The procedure below returns the grid location of the first empty location, which could be `0, 0` all the way to `8, 8`. It returns `-1, -1` if there are no empty locations.

In [8]:
def findNextCellToFill(grid):
    """Find coordinates of an empty cell in a Sudoku grid,
    or return -1, -1 if there are no empty cells."""
    for i in range(9):
        for j in range(9):
            if grid[i][j] == 0:
                return i, j
    return -1, -1

The procedure `isValid` below embodies the rules of Sudoku.  It takes a partially filled in Sudoku puzzle grid, and a new entry `e` at `grid[i, j]`, and checks whether filling in this entry violates any of the rules.

In [9]:
def isValid(grid, i, j, e):
    """Would setting grid[i,j] = e satisfy all Sudoku constraints?"""
    rowOk = all(e != grid[i][x] for x in range(9))
    if not rowOk: return False
    columnOk = all(e != grid[x][j] for x in range(9))
    if not columnOk: return False
    secTopX, secTopY = 3 * (i//3), 3 * (j//3)
    for x in range(secTopX, secTopX+3):
        for y in range(secTopY, secTopY+3):
            if grid[x][y] == e:
                return False
    return True

The procedure first checks that each row does not already have an element numbered with `e` on Line 3. It does this by using the `all` operator. Line 3 is equivalent to iterating through `grid[i][x]` for `x` from `0` through `8` and returning `False` if any entry is equal to `e`, and returning `True` otherwise. If this check passes, the column corresponding to `j` is checked on Line 5. If the column check passes, we determine the sector that `grid[i][j]` corresponds to (Line 7).  We then check whether any of the existing numbers in the sector are equal to `e` in Lines 8-11.

Note that `isValid` is like N-queens `noConflicts` in that it checks only whether a new entry violates Sudoku rules, since it focuses on the row, column, and sector of the new entry. If say `i = 2`, `j = 2`, `e = 2`, it does not check that the `i`th row does not already have two `3`’s on it, for instance. It is therefore important to call `isValid` each time an entry is made and `solveSudoku` does that.

Finally, here is a simple printing procedure so we can output something that (sort of) looks like a solved Sudoku puzzle:

In [10]:
def printSudoku(grid):
    """Display Sudoku grid, showing subgrid boundaries"""
    numrow = 0
    for row in grid:
        if numrow % 3 == 0 and numrow != 0:
            print(' ')
        print(row[0:3], ' ', row[3:6], ' ', row[6:9])
        numrow += 1

Line 5 prints a space to create a line spacing after three rows are printed. Remember that each `print` statement produces output on a different line if we do not set `end = ''`.

We are now ready to run the Sudoku solver. Here’s an input puzzle given as a two-dimensional array/list:

In [11]:
input = [[5, 1, 7, 6, 0, 0, 0, 3, 4],
         [2, 8, 9, 0, 0, 4, 0, 0, 0],
         [3, 4, 6, 2, 0, 5, 0, 9, 0],
         [6, 0, 2, 0, 0, 0, 0, 1, 0],
         [0, 3, 8, 0, 0, 6, 0, 4, 7],
         [0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 9, 0, 0, 0, 0, 0, 7, 8],
         [7, 0, 3, 4, 0, 0, 5, 6, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0]]

backtracks = 0
solveSudoku(input)
printSudoku(input)
print()
print(backtracks, 'backtracks')

[5, 1, 7]   [6, 9, 8]   [2, 3, 4]
[2, 8, 9]   [1, 3, 4]   [7, 5, 6]
[3, 4, 6]   [2, 7, 5]   [8, 9, 1]
 
[6, 7, 2]   [8, 4, 9]   [3, 1, 5]
[1, 3, 8]   [5, 2, 6]   [9, 4, 7]
[9, 5, 4]   [7, 1, 3]   [6, 8, 2]
 
[4, 9, 5]   [3, 6, 2]   [1, 7, 8]
[7, 2, 3]   [4, 8, 1]   [5, 6, 9]
[8, 6, 1]   [9, 5, 7]   [4, 2, 3]

579 backtracks


Check to make sure the puzzle was solved correctly.

The second example puzzle shown below is based on the first puzzle, but with a few numbers removed. This makes the puzzle harder, which can be seen by the much larger number of backtracks.

In [12]:
input2 = [[5, 1, 7, 6, 0, 0, 0, 3, 4],
          [0, 8, 9, 0, 0, 4, 0, 0, 0],
          [3, 0, 6, 2, 0, 5, 0, 9, 0],
          [6, 0, 0, 0, 0, 0, 0, 1, 0],
          [0, 3, 0, 0, 0, 6, 0, 4, 7],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 9, 0, 0, 0, 0, 0, 7, 8],
          [7, 0, 3, 4, 0, 0, 5, 6, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0]]

backtracks = 0
solveSudoku(input2)
printSudoku(input2)
print()
print(backtracks, 'backtracks')

[5, 1, 7]   [6, 9, 8]   [2, 3, 4]
[2, 8, 9]   [1, 3, 4]   [7, 5, 6]
[3, 4, 6]   [2, 7, 5]   [8, 9, 1]
 
[6, 7, 2]   [8, 4, 9]   [3, 1, 5]
[1, 3, 8]   [5, 2, 6]   [9, 4, 7]
[9, 5, 4]   [7, 1, 3]   [6, 8, 2]
 
[4, 9, 5]   [3, 6, 2]   [1, 7, 8]
[7, 2, 3]   [4, 8, 1]   [5, 6, 9]
[8, 6, 1]   [9, 5, 7]   [4, 2, 3]

6363 backtracks


## Implications During Recursive Search

The basic solver does not perform the implications that we described in determining the position for the 8 in our very first Sudoku example. The same technique can be expanded by using information from perpendicular rows and columns. Let’s see where we can place a 1 in the top right box in the example below. Row 1 and row 2 contain 1’s, which leaves two empty squares at the bottom of our focus box. However, square g4 also contains 1, so no additional 1 is allowed in column g. Hence square i3 is the only place left for 1.

<div style="text-align:center"><img style="display:inline-block" width="40%" src="img/sudoku3.png">$\qquad$<img style="display:inline-block" width="40%" src="img/sudoku4.png"></div>

How can the recursive Sudoku solver be augmented to perform these implications?
 
We will show how to augment our solver to perform this implication and see how much more efficient the solver becomes. We can do this by measuring the number of backtracks with and without implications. Implications more quickly determine whether a particular assignment of values to empty squares is correct or not.

There are several changes that need to be made to the solver in order to correctly implement this optimization.  To be clear, this optimization is thought of as an implication because the current state of the grid implies a position for the 1 in the example above. There can be one or more implications that can be made once a particular grid location is assigned a value.  The recursive search code in the optimized solver needs to be slightly different. You can find the code in `sudoku-opt.py`.

In [13]:
backtracks = 0

def solveSudokuOpt(grid):
    """Try to solve a Sudoku puzzle in-place, and return whether successful.
    grid is a list of lists of integers between 0 and 9, where 0 means unfilled,
    and each top-level list corresponds to a row of 9 cells."""
    global backtracks  
    i, j = findNextCellToFill(grid)
    if i == -1: # no more empty cells
        return True
    for e in range(1, 10):
        if isValid(grid, i, j, e):
            impl = makeImplications(grid, i, j, e)  #NEW
            if solveSudokuOpt(grid):
                return True
            backtracks += 1
            undoImplications(grid, impl)            #NEW
    return False

The only changes are on Lines 13 and 17. On Line 13, not only are we filling in the `grid[i][j]` entry with `e`, but we are also making implications and filling in other grid locations. All of these have to be “remembered” in the implication list `impl`.  On Line 17, we have to undo all of the changes made to the grid because the `grid[i][j] = e` guess was incorrect.

Storing the assignment and implications performed so we can roll them all back if the assignment does not work is important for correctness — else we might not explore the entire search space and therefore not find a solution.  To understand this, look at the figure below.

<img width="50%" src="img/implications.png">

Think of A, B and C above as being grid locations and assume that we only have two numbers 1 and 2 that are possible entries. (We have a simplified situation for illustration purposes.) Suppose we assign A = 1, B = 1, and then C = 2 is implied.  After exploring the A = 1, B = 1 branch fully, we backtrack to A = 1, B = 2.  Here, we need to explore C = 1 and C = 2 as in the picture to the left, not just C = 2 as shown on the picture to the right.  What might happen is that C is still set to 2 in the B = 2 branch and we, in effect, only explore the B = 2, C = 2 branch. So we need to roll back all the implications associated with an assignment.

The procedure `undoImplications` is short and is shown below.

In [14]:
def undoImplications(grid, impl):
    """Reset all implied grid cells listed in impl to 0"""
    for i, j, e in impl:
        grid[i][j] = 0

`impl` is a list of 3-tuples, where each 3-tuple is of the form `(i, j, e)` meaning that `grid[i][j] = e`. In `undoImplications`, we don’t care about the third item `e` since we want to empty out the entry.

`makeImplications` is more involved since it performs significant analysis. The pseudocode for `makeImplications` is below. The line numbers are for the Python code that is shown after the pseudocode.

* For each sector (subgrid):
  * Find missing elements in the sector (Lines 10-14)
  * Attach set of missing elements to each empty square in sector (Lines 15-18)
  * For each empty square S in sector: (Line 19)
    * Subtract all elements on S’s row from missing elements set (Lines 20-23)
    * Subtract all elements on S’s column from missing elements set (Lines 24-27)
    * If missing elements set is a *single* value then: (Line 28)
      * Missing square value can be implied to be that value (Lines 29-32)

In [15]:
sectors = [[0, 3, 0, 3], [3, 6, 0, 3], [6, 9, 0, 3],
           [0, 3, 3, 6], [3, 6, 3, 6], [6, 9, 3, 6],
           [0, 3, 6, 9], [3, 6, 6, 9], [6, 9, 6, 9]]

def makeImplications(grid, i, j, e):
    """Set grid[i][j] = e and fill in all implied cell values.
    Return a list of (x,y,v) triples for each grid[x][y] set to v.
    """
    grid[i][j] = e
    impl = [(i, j, e)]
    for sector in sectors:
        sectblanks = []
        vset = {1, 2, 3, 4, 5, 6, 7, 8, 9}
        for x in range(sector[0], sector[1]):
            for y in range(sector[2], sector[3]):
                if grid[x][y] != 0:
                    vset.remove(grid[x][y])
        for x in range(sector[0], sector[1]):
            for y in range(sector[2], sector[3]):
                if grid[x][y] == 0:
                    sectblanks.append([x, y, vset.copy()])
        for blankx, blanky, left in sectblanks:
            rowv = set(grid[blankx][y] for y in range(9))
            left = left.difference(rowv)          
            colv = set(grid[x][blanky] for x in range(9))
            left = left.difference(colv)                         
            if len(left) == 1:
                val = left.pop()
                if isValid(grid, blankx, blanky, val):
                    grid[blankx][blanky] = val
                    impl.append((blankx, blanky, val))              
    return impl

Line 1 declares variables that give the grid indices of each of the 9 sectors.  For example, the middle sector 4 varies from 3 to 5 inclusive in the x and y coordinates. This is helpful in ranging over the grid but staying within a sector. 

This code uses the `set` data structure in Python.  An empty set is declared using `set()` as opposed to an empty list which is declared as `[]`. (We cannot use `{}` because that defines a dictionary.)  A set cannot have repeated elements: if we included a number, say 1, twice in the declaration of a set, it would only be included once in the set, so `V = {1, 1, 2}` is the same as `V = {1, 2}`.

Line 13 declares a set `vset` that contains numbers 1 through 9.  In Lines 13-17, we go through the elements in the sector and remove these elements from `vset` using the remove function. We wish to append this missing-element set to each empty square and hence we create a list `sectinfo` of 3-tuples. Each 3-tuple has the x, y coordinates of the empty square in the sector, and a copy of the set of missing elements in the sector. We need to make copies of sets because these copies will diverge in their membership later in the algorithm.

For each empty square in the sector, we look at the corresponding 3-tuple in `sectinfo` (Line 22). The elements that are in the corresponding row are removed from the missing element set given by `sin[2]`, the third element of the 3-tuple by using the set difference function (Line 26).  Similarly, for the column associated with the empty square. The remaining elements are stored in the set `left`.

If the set `left` has cardinality 1 (Line 31), we may have an implication. Why are we not guaranteed an implication? The way we have written the code, we compute the missing elements for each sector, and try to find implications for each empty square in the sector. The very first implication will hold, but once we make one particular implication, the sector changes as does the missing elements set.  So further implications computed using stale missing elements information may not be valid.  This is why we check whether the implication violates the rules of Sudoku on Line 33 prior to including it in the implication list `impl`. 

This optimization shrinks the number of backtracks down from 579 to 10 for the puzzle `input` and from 6,363 to 33 for puzzle `input2`. Of course, from a standpoint of computer time usage, both versions run in fractions of a second! This is one of the reasons why we included the functionality of counting backtracks in the code so you could see that the optimizations do help reduce the guessing required.

In [16]:
input = [[5, 1, 7, 6, 0, 0, 0, 3, 4],
         [2, 8, 9, 0, 0, 4, 0, 0, 0],
         [3, 4, 6, 2, 0, 5, 0, 9, 0],
         [6, 0, 2, 0, 0, 0, 0, 1, 0],
         [0, 3, 8, 0, 0, 6, 0, 4, 7],
         [0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 9, 0, 0, 0, 0, 0, 7, 8],
         [7, 0, 3, 4, 0, 0, 5, 6, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0]]

backtracks = 0
solveSudokuOpt(input)
printSudoku(input)
print()
print(backtracks, 'backtracks')

[5, 1, 7]   [6, 9, 8]   [2, 3, 4]
[2, 8, 9]   [1, 3, 4]   [7, 5, 6]
[3, 4, 6]   [2, 7, 5]   [8, 9, 1]
 
[6, 7, 2]   [8, 4, 9]   [3, 1, 5]
[1, 3, 8]   [5, 2, 6]   [9, 4, 7]
[9, 5, 4]   [7, 1, 3]   [6, 8, 2]
 
[4, 9, 5]   [3, 6, 2]   [1, 7, 8]
[7, 2, 3]   [4, 8, 1]   [5, 6, 9]
[8, 6, 1]   [9, 5, 7]   [4, 2, 3]

10 backtracks


In [17]:
input2 = [[5, 1, 7, 6, 0, 0, 0, 3, 4],
          [0, 8, 9, 0, 0, 4, 0, 0, 0],
          [3, 0, 6, 2, 0, 5, 0, 9, 0],
          [6, 0, 0, 0, 0, 0, 0, 1, 0],
          [0, 3, 0, 0, 0, 6, 0, 4, 7],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 9, 0, 0, 0, 0, 0, 7, 8],
          [7, 0, 3, 4, 0, 0, 5, 6, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0]]

backtracks = 0
solveSudokuOpt(input2)
printSudoku(input2)
print()
print(backtracks, 'backtracks')

[5, 1, 7]   [6, 9, 8]   [2, 3, 4]
[2, 8, 9]   [1, 3, 4]   [7, 5, 6]
[3, 4, 6]   [2, 7, 5]   [8, 9, 1]
 
[6, 7, 2]   [8, 4, 9]   [3, 1, 5]
[1, 3, 8]   [5, 2, 6]   [9, 4, 7]
[9, 5, 4]   [7, 1, 3]   [6, 8, 2]
 
[4, 9, 5]   [3, 6, 2]   [1, 7, 8]
[7, 2, 3]   [4, 8, 1]   [5, 6, 9]
[8, 6, 1]   [9, 5, 7]   [4, 2, 3]

33 backtracks


## More Improvement

We can improve our optimized Sudoku solver as follows. Each time we discover an implication, the grid changes, and we may find other implications. In fact, this is the way humans solve Sudoku puzzles. Our optimized solver goes through all the sectors trying to find implications, and then stops. If we find an implication in one “pass” through the grid sectors, we could try repeating the entire process (Lines 8-33) until we can’t find implications, i.e., can’t add to our data structure `impl`.  The file `sudoku-more-opt.py` has the improved solver. For the Sudoku puzzle `input2`, the number of backtracks reduces from 33 to 2.

## Difficulty of Sudoku Puzzles

A Finnish mathematician Arto Inkala claimed in 2006 claimed he had created the world’s hardest Sudoku puzzle and followed it up in 2010 with a claim of an even harder puzzle. The first puzzle takes the unoptimized solver 335,578 backtracks and the second 9,949 backtracks! The solver finds solutions in a matter of seconds. To be fair, Inkala was predicting human difficulty. Here’s Inkala’s 2010 puzzle below.

<img width="50%" src="img/sudoku5.png">

Peter Norvig has written Sudoku solvers that use constraint programming techniques significantly more sophisticated than the simple implications we have presented here. As a result the amount of backtracking required even for difficult puzzles is quite small.

We suggest you find Sudoku puzzles with different levels of difficulty, from easy to very hard and explore how the number of backtracks required in the basic solver and the optimized solver change as the level of difficulty increases.  You might be surprised by what you find!